<a href="https://colab.research.google.com/github/farfaness/Spark/blob/master/Dataframe_Databricks_Community_Edition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from pyspark.sql import SparkSession # Importer l'objet SparkSession (inutile dans le cas du Notebook Databricks)

In [0]:
# On instantie la SparkSession : cet objet est déja instantié dans Databricks.
# Il va nous suivre tout au long de l'application
# Les options de configurations sont infinies : elles peuvent toucher le contexte d'execution, 
# le nombre de noeuds utilisés... 
# Ici nous précisons le minimum : Le nom de l'application et le master, c'est à dire l'allocateur de ressources.
# En disant local on va utiliser l'allocateur de ressources interne à Spark : Spark Standalone. 
# Si vous lancez votre job Spark sur un cluster Hadoop, vous pouvez utiliser yarn
spark = SparkSession.builder.appName("First App").master("local").getOrCreate()

In [0]:
# Le sparkContext est l'outil qui permet d'explorer le plus bas niveau API de spark : la RDD
sc = spark.sparkContext

#### Charger la donnée

In [0]:
bateaux = spark.read.parquet("/FileStore/tables/2c443e0c8b99190f_c98fa37700000000_457998991_data_0__1__1_-c0514.parq")

In [0]:
bateaux.show()

+---------+---------+----+------------------+-------------------+-------------+-----+-------+--------------------+
id_vessel| mmsi|type| lat| lon|timestamp_pos|speed|heading| payload|
+---------+---------+----+------------------+-------------------+-------------+-----+-------+--------------------+
 4440| 0| 35| null| null|1416006922000| null| null|[5C 73 3A 72 4F 5...|
 121165|412417168| 30|28.596466666666668| 122.41604|1416009251000| 7.8| null|[5C 73 3A 72 4F 5...|
 124436|412688540| 38| 39.01532| 121.70704|1416009253000| 0.0| null|[5C 73 3A 72 4F 5...|
 123311|563610000| 70| 49.08342666666667| -123.345925|1416009253000| 13.3| 209|[5C 73 3A 72 4F 5...|
 63048|413807776| 70| 22.217245| 114.32744166666667|1416009253000| 5.6| null|[5C 73 3A 72 4F 5...|
 60202|244690585| 51| 52.18333833333333| 4.996126666666667|1416009251000| 0.0| null|[5C 73 3A 72 4F 5...|
 18884|412464280| 52|22.565291666666667| 114.273425|1416009253000| 3.8| 268|[5C 73 3A 72 4F 5...|
 6899|244710042| 80| 51.81129| 4.611525|1416009253000| 0.0| null|[5C 73 3A 72 4F 5...|
 65727|636016121| 80| 12.4185| -70.130165|1416009253000| 0.0| 93|[5C 73 3A 72 4F 5...|
 33529|258310000| 89| 63.43951166666667| 10.350478333333333|1416009251000| 0.0| 353|[5C 73 3A 72 4F 5...|
 35619|228284800| 52| 49.47528| 0.132895|1416009253000| 0.0| 278|[5C 73 3A 72 4F 5...|
 41040|636091848| 71|27.900001666666668|-13.668798333333333|1416009253000| 16.3| 225|[5C 73 3A 72 4F 5...|
 113862|412565686| 30| 39.71163166666667| 119.82036|1416009253000| 0.0| null|[5C 73 3A 72 4F 5...|
 6912|357361000| 70|31.990001666666668| 120.63421|1416009251000| 0.1| 277|[5C 73 3A 72 4F 5...|
 103343|432620000| 70|-3.715298333333333| 114.4597|1416006921000| 0.0| 21|[5C 73 3A 72 4F 5...|
 27883|211494070| 80| 51.83454| 4.48183|1416009253000| 7.7| null|[5C 73 3A 72 4F 5...|
 12186|432876000| 70| 36.966695| -76.42965166666667|1416009253000| 0.0| 42|[5C 73 3A 72 4F 5...|
 184842|352841000| 89| 42.56032666666667| 141.71708166666667|1416009251000| 3.1| 310|[5C 73 3A 72 4F 5...|
 31003|244690618| 79| 51.86452| 6.063571666666666|1416009253000| 0.0| null|[5C 73 3A 72 4F 5...|
 31003|244690618| 79| 51.86452| 6.063571666666666|1416009253000| 0.0| null|[5C 73 3A 72 4F 5...|
+---------+---------+----+------------------+-------------------+-------------+-----+-------+--------------------+
only showing top 20 rows

#### Afficher le nombre d'éléments du DF

In [0]:
bateaux.count()

Out[ 13 ]: 314943

#### Afficher le type des éléments du DF

In [0]:
bateaux.schema

Out[ 14 ]: StructType(List(StructField(id_vessel,LongType,true),StructField(mmsi,LongType,true),StructField(type,IntegerType,true),StructField(lat,DoubleType,true),StructField(lon,DoubleType,true),StructField(timestamp_pos,LongType,true),StructField(speed,DoubleType,true),StructField(heading,IntegerType,true),StructField(payload,BinaryType,true)))

#### Afficher le schéma du DF

In [0]:
bateaux.printSchema()

root
-- id_vessel: long (nullable = true)
-- mmsi: long (nullable = true)
-- type: integer (nullable = true)
-- lat: double (nullable = true)
-- lon: double (nullable = true)
-- timestamp_pos: long (nullable = true)
-- speed: double (nullable = true)
-- heading: integer (nullable = true)
-- payload: binary (nullable = true)

#### Ordonnez les bateaux par vitesse

In [0]:
# Rappel : on ne peut modifier un RDD, ni une DataFrame. Ici nous avons donc créer une nouvelle DF et détruit l'ancienne
# Si on veut garder la trace de l'ancienne il est important de charger la nouvelle dans une autre variable
bateaux = bateaux.orderBy("speed")

#### Pour disposer d'une date plus lisible, rajoutez la colonne "timestamp_readable"
#### Pour cela, il vous faudra d'abord transformer le timestamp en secondes (timestamp_pos est en ms) et ensuite convertir en format date

In [0]:
bateaux_date = bateaux.withColumn("timestamp_readable", (bateaux["timestamp_pos"]/1000).cast("timestamp"))

In [0]:
bateaux_date.show()

+---------+---------+----+------------------+-------------------+-------------+-----+-------+--------------------+-------------------+
id_vessel| mmsi|type| lat| lon|timestamp_pos|speed|heading| payload| timestamp_readable|
+---------+---------+----+------------------+-------------------+-------------+-----+-------+--------------------+-------------------+
 70244|710012740| 90| null| null|1416008896000| null| 195|[5C 73 3A 72 4F 5...|2014-11-14 23:48:16|
 96190|250104640| 30|52.171166666666664| -6.587388333333333|1416007672000| null| 351|[5C 73 3A 72 4F 5...|2014-11-14 23:27:52|
 96190|250104640| 30|52.171146666666665| -6.587241666666666|1416009253000| null| 351|[5C 73 3A 72 4F 5...|2014-11-14 23:54:13|
 86972|367136250| 54| null| null|1416009378000| null| null|[5C 73 3A 72 4F 5...|2014-11-14 23:56:18|
 116462|244615906| 99| null| null|1416009533000| null| null|[5C 73 3A 72 4F 5...|2014-11-14 23:58:53|
 4440| 0| 35| null| null|1416006922000| null| null|[5C 73 3A 72 4F 5...|2014-11-14 23:15:22|
 17021|244670016| 79| null| null|1416009536000| null| null|[5C 73 3A 72 4F 5...|2014-11-14 23:58:56|
 126459|366858820| 69| 39.285665| -76.61075|1416006797000| null| null|[5C 73 3A 72 4F 5...|2014-11-14 23:13:17|
 70391|412358940| 70|29.935871666666667| 121.68577666666667|1416006456000| null| null|[5C 73 3A 72 4F 5...|2014-11-14 23:07:36|
 128314|273196000| 10| 47.13505333333333| 38.961803333333336|1416006797000| null| null|[5C 73 3A 72 4F 5...|2014-11-14 23:13:17|
 9874|273445400| 52| 44.72857166666667| 37.78629333333333|1416006451000| null| null|[5C 73 3A 72 4F 5...|2014-11-14 23:07:31|
 61519|413300330| 70| 37.66363166666667| 120.309705|1416007303000| null| 93|[5C 73 3A 72 4F 5...|2014-11-14 23:21:43|
 60851|431301253| 52| 34.33177166666667| 133.03930166666666|1416006448000| null| null|[5C 73 3A 72 4F 5...|2014-11-14 23:07:28|
 159160|413460940|null|24.432866666666666| 117.88066666666667|1416007596000| null| null|[5C 73 3A 72 4F 5...|2014-11-14 23:26:36|
 129426|413888231| 70|38.856276666666666| 118.03631166666666|1416008637000| null| null|[5C 73 3A 72 4F 5...|2014-11-14 23:43:57|
 38742|273353280| 52| 42.75065333333333| 133.08373833333334|1416006797000| null| 21|[5C 73 3A 72 4F 5...|2014-11-14 23:13:17|
 192782|755002986| 52|-32.94766666666667|-60.595751666666665|1416009361000| null| 199|[5C 73 3A 72 4F 5...|2014-11-14 23:56:01|
 19409|413400540| 61| 29.993| 122.04246166666667|1416007303000| null| 73|[5C 73 3A 72 4F 5...|2014-11-14 23:21:43|
 54103|413044990| 55|29.952031666666667| 121.71950666666666|1416007668000| null| null|[5C 73 3A 72 4F 5...|2014-11-14 23:27:48|
 54103|413044990| 55|29.952031666666667| 121.71950666666666|1416007668000| null| null|[5C 73 3A 72 4F 5...|2014-11-14 23:27:48|
+---------+---------+----+------------------+-------------------+-------------+-----+-------+--------------------+-------------------+
only showing top 20 rows

#### Il est possible de créer un nouveau dataframe contenant des statistiques sur un dataframe

In [0]:
description_df = bateaux.describe()
description_df.show()

+-------+------------------+--------------------+-----------------+-----------------+-------------------+--------------------+-----------------+------------------+
summary| id_vessel| mmsi| type| lat| lon| timestamp_pos| speed| heading|
+-------+------------------+--------------------+-----------------+-----------------+-------------------+--------------------+-----------------+------------------+
 count| 314943| 314943| 306227| 314734| 314734| 314943| 313477| 165448|
 mean| 84044.48684365107| 3.710984933372325E8|62.17296319397049|31.64549660628215| 43.891339197629215|1.416007766095668...|2.968054115612944| 175.7893416662637|
 stddev|104847.51303519521|1.2919476616551399E8|21.44858919019196|22.63829025259374| 78.35610533433496| 1033950.6197545934|5.251255455795127|106.90464686871879|
 min| 1| 0| 1| -88.799435|-179.97899833333332| 1416006001000| 0.0| 0|
 max| 1454230| 1063852744| 255|86.93066166666667| 179.9999| 1416009595000| 102.0| 359|
+-------+------------------+--------------------+-----------------+-----------------+-------------------+--------------------+-----------------+------------------+

#### Filtrer le dataframe bateaux sur un bateau en particulier

In [0]:
monBateau = bateaux.filter("id_vessel = 28052")
monBateau.show()

+---------+---------+----+------------------+------------------+-------------+-----+-------+--------------------+
id_vessel| mmsi|type| lat| lon|timestamp_pos|speed|heading| payload|
+---------+---------+----+------------------+------------------+-------------+-----+-------+--------------------+
 28052|345971158| 50|18.437848333333335|-93.18948666666667|1416007969000| 0.0| null|[5C 73 3A 72 4F 5...|
 28052|345971158| 50|18.437856666666665| -93.189515|1416009589000| 0.0| null|[5C 73 3A 72 4F 5...|
 28052|345971158| 50|18.437848333333335| -93.189505|1416008808000| 0.0| null|[5C 73 3A 72 4F 5...|
 28052|345971158| 50|18.437866666666668|-93.18950666666667|1416006418000| 0.0| null|[5C 73 3A 72 4F 5...|
 28052|345971158| 50|18.437866666666668|-93.18950333333333|1416007139000| 0.0| null|[5C 73 3A 72 4F 5...|
+---------+---------+----+------------------+------------------+-------------+-----+-------+--------------------+

#### Le nombre de bateaux avec une vitesse == 0 et la fraction que cela représente sur par rapport au nombre total

#### Les types distincts de bateaux

In [0]:
bateaux[["type"]].dropDuplicates().show()

+----+
type|
+----+
 148|
 31|
 85|
 137|
 65|
 53|
 255|
 133|
 78|
 155|
 34|
 193|
 115|
 101|
 81|
 210|
 183|
 28|
 76|
 26|
+----+
only showing top 20 rows

#### On affiche la moyenne de la vitesse des bateaux sans tenir compte des bateaux a l'arret

In [0]:
enmarche = bateaux.filter("speed !=0 ").groupBy("id_vessel").mean("speed")

In [0]:
enmarche.show()

+---------+-------------------+
id_vessel| avg(speed)|
+---------+-------------------+
 15237| 0.125|
 9233| 0.1|
 30428|0.18000000000000002|
 13452|0.18000000000000002|
 92741| 0.1|
 9715| 0.1|
 133382|0.13333333333333333|
 29089|0.16666666666666666|
 86941| 0.26|
 84272| 0.1|
 7225| 0.1|
 54596| 0.1|
 55845| 0.1|
 18147|0.21428571428571427|
 188945|0.10000000000000002|
 55474| 0.1|
 60691| 0.1|
 55604| 0.1|
 54536| 0.1|
 74783| 0.1|
+---------+-------------------+
only showing top 20 rows

#### On affiche la date min puis  la date max pour chaque bateau

In [0]:
bateaux_date.groupBy("id_vessel").agg({"timestamp_readable": "min"}).show()

+---------+-----------------------+
id_vessel|min(timestamp_readable)|
+---------+-----------------------+
 128314| 2014-11-14 23:13:17|
 61888| 2014-11-14 23:05:25|
 30562| 2014-11-14 23:25:32|
 206429| 2014-11-14 23:10:58|
 115909| 2014-11-14 23:11:31|
 54536| 2014-11-14 23:04:09|
 2927| 2014-11-14 23:01:47|
 25479| 2014-11-14 23:04:06|
 96165| 2014-11-14 23:12:46|
 49754| 2014-11-14 23:07:50|
 38510| 2014-11-14 23:21:19|
 64873| 2014-11-14 23:00:34|
 98424| 2014-11-14 23:10:24|
 63212| 2014-11-14 23:07:08|
 100852| 2014-11-14 23:06:56|
 39527| 2014-11-14 23:09:43|
 8440| 2014-11-14 23:09:43|
 69720| 2014-11-14 23:03:34|
 10959| 2014-11-14 23:01:51|
 22165| 2014-11-14 23:01:39|
+---------+-----------------------+
only showing top 20 rows

In [0]:
bateaux_date.groupBy("id_vessel").agg({"timestamp_readable": "max"}).show()

+---------+-----------------------+
id_vessel|max(timestamp_readable)|
+---------+-----------------------+
 128314| 2014-11-14 23:59:37|
 61888| 2014-11-14 23:38:53|
 30562| 2014-11-14 23:25:32|
 206429| 2014-11-14 23:43:59|
 115909| 2014-11-14 23:44:01|
 54536| 2014-11-14 23:55:08|
 2927| 2014-11-14 23:43:15|
 25479| 2014-11-14 23:46:06|
 96165| 2014-11-14 23:52:46|
 49754| 2014-11-14 23:58:00|
 38510| 2014-11-14 23:48:20|
 64873| 2014-11-14 23:52:52|
 98424| 2014-11-14 23:58:48|
 63212| 2014-11-14 23:46:15|
 100852| 2014-11-14 23:51:56|
 39527| 2014-11-14 23:49:53|
 8440| 2014-11-14 23:39:44|
 69720| 2014-11-14 23:49:43|
 10959| 2014-11-14 23:55:22|
 22165| 2014-11-14 23:56:09|
+---------+-----------------------+
only showing top 20 rows

#### Créer un dataframe avec l'id, la vitesse et le timestamp readable

In [0]:
spd = bateaux_date.select(["id_vessel","speed","timestamp_readable"])

#### Créer une colonne mois pour le mois en cours

In [0]:
from pyspark.sql.functions import month
spd = spd.withColumn("month", month(bateaux_date["timestamp_readable"]))
spd.show()

+---------+-----+-------------------+-----+
id_vessel|speed| timestamp_readable|month|
+---------+-----+-------------------+-----+
 70244| null|2014-11-14 23:48:16| 11|
 96190| null|2014-11-14 23:27:52| 11|
 96190| null|2014-11-14 23:54:13| 11|
 86972| null|2014-11-14 23:56:18| 11|
 116462| null|2014-11-14 23:58:53| 11|
 4440| null|2014-11-14 23:15:22| 11|
 17021| null|2014-11-14 23:58:56| 11|
 126459| null|2014-11-14 23:13:17| 11|
 70391| null|2014-11-14 23:07:36| 11|
 128314| null|2014-11-14 23:13:17| 11|
 9874| null|2014-11-14 23:07:31| 11|
 61519| null|2014-11-14 23:21:43| 11|
 60851| null|2014-11-14 23:07:28| 11|
 159160| null|2014-11-14 23:26:36| 11|
 129426| null|2014-11-14 23:43:57| 11|
 38742| null|2014-11-14 23:13:17| 11|
 192782| null|2014-11-14 23:56:01| 11|
 19409| null|2014-11-14 23:21:43| 11|
 54103| null|2014-11-14 23:27:48| 11|
 54103| null|2014-11-14 23:27:48| 11|
+---------+-----+-------------------+-----+
only showing top 20 rows

#### Ajouter une colonne boolean à True si le bateau est en mouvement

In [0]:
spd = spd.withColumn("BooleanCol", spd.speed != 0)

#### Enregistrer la table

In [0]:
bateaux.createOrReplaceTempView("tab_bateaux")

#### Afficher par type, le nombre de bateaux, la vitesse max et la moyenne de la vitesse, ordonnés par nombre de bateaux (requete SQL)

In [0]:
spark.sql("""SELECT type, max(speed), avg(speed), count(*) as num 
FROM tab_bateaux 
GROUP BY type 
ORDER BY count(*)""").show()

+----+----------+-------------------+---+
type|max(speed)| avg(speed)|num|
+----+----------+-------------------+---+
 232| 0.0| 0.0| 1|
 236| 8.5| 8.5| 1|
 240| 4.1| 3.9499999999999997| 2|
 113| 0.0| 0.0| 2|
 202| 0.0| 0.0| 2|
 212| 0.1| 0.05| 2|
 229| 3.1| 1.85| 2|
 119| 0.0| 0.0| 2|
 190| 0.0| 0.0| 2|
 199| 4.0| 3.8333333333333335| 3|
 149| 5.1| 1.8666666666666665| 3|
 244| 0.0| 0.0| 3|
 142| 0.0| 0.0| 3|
 137| 9.5| 9.233333333333333| 3|
 158| 0.0| 0.0| 3|
 120| 0.1|0.07500000000000001| 4|
 107| 0.0| 0.0| 4|
 177| 0.0| 0.0| 4|
 145| 0.0| 0.0| 4|
 171| 4.5| 2.15| 4|
+----+----------+-------------------+---+
only showing top 20 rows

### Mettons nos compétences au service d'un cas d'usage un peu plus intéressant.
Nous allons prendre un bateau de notre choix et nous allons tenter de calculer quels sont les bateaux qu'il a pu croiser sur sa route

#### Créer un nouveau dataframe basé sur le DF bateaux et filtré uniquement sur id_vessel de votre choix.
#### Débarassez vous également de la colonne "payload" avec la méthode df.drop("colonne")

In [0]:
monBateau.show()

+---------+---------+----+------------------+------------------+-------------+-----+-------+--------------------+
id_vessel| mmsi|type| lat| lon|timestamp_pos|speed|heading| payload|
+---------+---------+----+------------------+------------------+-------------+-----+-------+--------------------+
 28052|345971158| 50|18.437848333333335|-93.18948666666667|1416007969000| 0.0| null|[5C 73 3A 72 4F 5...|
 28052|345971158| 50|18.437856666666665| -93.189515|1416009589000| 0.0| null|[5C 73 3A 72 4F 5...|
 28052|345971158| 50|18.437848333333335| -93.189505|1416008808000| 0.0| null|[5C 73 3A 72 4F 5...|
 28052|345971158| 50|18.437866666666668|-93.18950666666667|1416006418000| 0.0| null|[5C 73 3A 72 4F 5...|
 28052|345971158| 50|18.437866666666668|-93.18950333333333|1416007139000| 0.0| null|[5C 73 3A 72 4F 5...|
+---------+---------+----+------------------+------------------+-------------+-----+-------+--------------------+

In [0]:
monBateau = monBateau.drop('payload')
monBateau.show()

+---------+---------+----+------------------+------------------+-------------+-----+-------+
id_vessel| mmsi|type| lat| lon|timestamp_pos|speed|heading|
+---------+---------+----+------------------+------------------+-------------+-----+-------+
 28052|345971158| 50|18.437848333333335|-93.18948666666667|1416007969000| 0.0| null|
 28052|345971158| 50|18.437856666666665| -93.189515|1416009589000| 0.0| null|
 28052|345971158| 50|18.437848333333335| -93.189505|1416008808000| 0.0| null|
 28052|345971158| 50|18.437866666666668|-93.18950666666667|1416006418000| 0.0| null|
 28052|345971158| 50|18.437866666666668|-93.18950333333333|1416007139000| 0.0| null|
+---------+---------+----+------------------+------------------+-------------+-----+-------+

#### Créer un deuxième dataframe basé sur le DF bateaux, avec les colonnes id_vessel, lat, lon et timestamp_pos
#### On transforme nos DF en rdd grâce à la fonction rdd (pas besoin)

In [0]:
bateaux2 = bateaux[["id_vessel", "lat", "lon", "timestamp_pos"]]
bateaux2.show()

+---------+------------------+-------------------+-------------+
id_vessel| lat| lon|timestamp_pos|
+---------+------------------+-------------------+-------------+
 70244| null| null|1416008896000|
 96190|52.171166666666664| -6.587388333333333|1416007672000|
 96190|52.171146666666665| -6.587241666666666|1416009253000|
 86972| null| null|1416009378000|
 116462| null| null|1416009533000|
 4440| null| null|1416006922000|
 17021| null| null|1416009536000|
 126459| 39.285665| -76.61075|1416006797000|
 70391|29.935871666666667| 121.68577666666667|1416006456000|
 128314| 47.13505333333333| 38.961803333333336|1416006797000|
 9874| 44.72857166666667| 37.78629333333333|1416006451000|
 61519| 37.66363166666667| 120.309705|1416007303000|
 60851| 34.33177166666667| 133.03930166666666|1416006448000|
 159160|24.432866666666666| 117.88066666666667|1416007596000|
 129426|38.856276666666666| 118.03631166666666|1416008637000|
 38742| 42.75065333333333| 133.08373833333334|1416006797000|
 192782|-32.94766666666667|-60.595751666666665|1416009361000|
 19409| 29.993| 122.04246166666667|1416007303000|
 54103|29.952031666666667| 121.71950666666666|1416007668000|
 54103|29.952031666666667| 121.71950666666666|1416007668000|
+---------+------------------+-------------------+-------------+
only showing top 20 rows

#### On calcule le nombre d'enregistrements associés à notre bateau
 
#### On calcule le nombre d'enregistrements du RDD de référence

In [0]:
print(monBateau.count(), bateaux2.count())
print((monBateau.count() * bateaux2.count()))

5 314943
1574715

##### Créer un RDD batEtude produit cartesien de notre RDD bateau et du RDD référentiel

In [0]:
from pyspark.sql.functions import col

monBateau = monBateau.select(col("id_vessel").alias("my_id_vessel"), col("lat").alias("my_lat"), col("lon").alias("my_lon"), col("timestamp_pos").alias("my_timestamp_pos"))
monBateau.show()

+------------+------------------+------------------+----------------+
my_id_vessel| my_lat| my_lon|my_timestamp_pos|
+------------+------------------+------------------+----------------+
 28052|18.437848333333335|-93.18948666666667| 1416007969000|
 28052|18.437856666666665| -93.189515| 1416009589000|
 28052|18.437848333333335| -93.189505| 1416008808000|
 28052|18.437866666666668|-93.18950666666667| 1416006418000|
 28052|18.437866666666668|-93.18950333333333| 1416007139000|
+------------+------------------+------------------+----------------+

In [0]:
batEtude = monBateau.crossJoin(bateaux2)
batEtude.count()

Out[ 54 ]: 1574715

##### On affiche la première ligne pour analyser son contenu
##### Astuce : n'oubliez pas que l'on est maintenant sur des RDD

In [0]:
batEtude.take(1)

Out[ 55 ]: [Row(my_id_vessel=28052, my_lat=18.437848333333335, my_lon=-93.18948666666667, my_timestamp_pos=1416007969000, id_vessel=4440, lat=None, lon=None, timestamp_pos=1416006922000)]

##### On pourrait s'amuser à compter le nombre de lignes totales du RDD.. mais c'est une action !
##### Il est possible que cela prenne un peu de temps selon la taille de votre RDD contenant un seul id_vessel

##### B4 - Filtrer notre produit cartésien pour éliminer les bateaux identiques à notre id_vessel choisi
##### Astuce : notre RDD batEtude est composé de tuples(bateau,bateauRef)

In [0]:
batEtude.columns

Out[ 56 ]: ['my_id_vessel',
 'my_lat',
 'my_lon',
 'my_timestamp_pos',
 'id_vessel',
 'lat',
 'lon',
 'timestamp_pos']

In [0]:
filtered = batEtude.filter(batEtude.id_vessel != batEtude.my_id_vessel)
filtered.count()

Out[ 60 ]: 1574690

##### On va également filtrer les informations des bateaux pour ne conserver que les enregistrements à +/- une heure (3600s)

In [0]:
#heure = 1416007969000
filtered2 = filtered.filter((filtered.my_timestamp_pos <= (filtered.timestamp_pos + 3600)) & (filtered.my_timestamp_pos >= (filtered.timestamp_pos - 3600)))
filtered2.count()

#filtered2 = filtered.filter(abs(filtered.my_timestamp_pos - filtered.timestamp_pos) < 3600)
#filtered2.count()

Out[ 108 ]: 3021

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import LongType
@F.udf(returnType=LongType())
def calculateDistanceInKilometer(userlat, userlon, otherlat, otherlon):
    AVERAGE_RADIUS_OF_EARTH_KM = 6371
    latDistance = math.radians(userlat - otherlat)
    lngDistance = math.radians(userlon - otherlon)
    sinLat = math.sin(latDistance / 2)
    sinLng = math.sin(lngDistance / 2)
    a = sinLat * sinLat + (math.cos(math.radians(userlat)) * math.cos(math.radians(otherlat))* sinLng * sinLng)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return int(AVERAGE_RADIUS_OF_EARTH_KM * c)

### Filtrer notre RDD en ne conservant que les bateaux pour lesquels la distance est inférieure à 1 km
La méthode calculateDistanceInKilometer(lat,lon,lat,long) renvoi la distance en kilomètres entre deux coordonnées

In [0]:
import math
import numpy as np
#filtered3 = filtered2.fillna(float(np.NaN))
filtered3 = filtered2.dropna()
filtered3.show()

+------------+------------------+------------------+----------------+---------+-------------------+-------------------+-------------+
my_id_vessel| my_lat| my_lon|my_timestamp_pos|id_vessel| lat| lon|timestamp_pos|
+------------+------------------+------------------+----------------+---------+-------------------+-------------------+-------------+
 28052|18.437848333333335|-93.18948666666667| 1416007969000| 44849| 22.167656666666666| 113.44122|1416007966000|
 28052|18.437848333333335|-93.18948666666667| 1416007969000| 71025| 21.929145| 113.2102|1416007966000|
 28052|18.437848333333335|-93.18948666666667| 1416007969000| 44849| 22.167656666666666| 113.44122|1416007966000|
 28052|18.437848333333335|-93.18948666666667| 1416007969000| 8120| 42.76967166666667| 133.016045|1416007969000|
 28052|18.437848333333335|-93.18948666666667| 1416007969000| 6750| 18.433628333333335| -93.18374333333334|1416007969000|
 28052|18.437848333333335|-93.18948666666667| 1416007969000| 61001| 51.262116666666664| 4.288133333333334|1416007967000|
 28052|18.437848333333335|-93.18948666666667| 1416007969000| 26393| 34.841181666666664| 128.42653833333333|1416007967000|
 28052|18.437848333333335|-93.18948666666667| 1416007969000| 18050| 68.08712166666666| 13.22675|1416007967000|
 28052|18.437848333333335|-93.18948666666667| 1416007969000| 70530| 41.922223333333335| -73.97139666666666|1416007969000|
 28052|18.437848333333335|-93.18948666666667| 1416007969000| 48025|-19.810033333333333| 34.83399|1416007969000|
 28052|18.437848333333335|-93.18948666666667| 1416007969000| 1833| 20.051415| 110.320075|1416007967000|
 28052|18.437848333333335|-93.18948666666667| 1416007969000| 1833| 20.051415| 110.320075|1416007967000|
 28052|18.437848333333335|-93.18948666666667| 1416007969000| 182002| 58.304226666666665| 11.698416666666667|1416007969000|
 28052|18.437848333333335|-93.18948666666667| 1416007969000| 160989| 62.85052| 7.141146666666667|1416007967000|
 28052|18.437848333333335|-93.18948666666667| 1416007969000| 69512| 42.77183333333333| 132.88871666666665|1416007967000|
 28052|18.437848333333335|-93.18948666666667| 1416007969000| 59012| 58.651833333333336| 9.140515|1416007967000|
 28052|18.437848333333335|-93.18948666666667| 1416007969000| 52012| 47.03621| 38.90099333333333|1416007967000|
 28052|18.437848333333335|-93.18948666666667| 1416007969000| 30554| 49.294756666666665|-123.13234833333334|1416007969000|
 28052|18.437848333333335|-93.18948666666667| 1416007969000| 82083| 27.83151833333333| 120.855315|1416007967000|
 28052|18.437848333333335|-93.18948666666667| 1416007969000| 217341| 27.830735| 120.98736166666667|1416007967000|
+------------+------------------+------------------+----------------+---------+-------------------+-------------------+-------------+
only showing top 20 rows

In [0]:
 filtered3.dtypes

Out[ 117 ]: [('my_id_vessel', 'bigint'),
 ('my_lat', 'double'),
 ('my_lon', 'double'),
 ('my_timestamp_pos', 'bigint'),
 ('id_vessel', 'bigint'),
 ('lat', 'double'),
 ('lon', 'double'),
 ('timestamp_pos', 'bigint')]

In [0]:
from pyspark.sql.types import FloatType
cols = ['my_lat', 'my_lon', 'lat', 'lon']

dataframe_temp = filtered3

for col_name in cols:
  dataframe_temp = dataframe_temp.withColumn(col_name, col(col_name).cast('float'))

dataframe_temp.dtypes  

Out[ 115 ]: [('my_id_vessel', 'bigint'),
 ('my_lat', 'float'),
 ('my_lon', 'float'),
 ('my_timestamp_pos', 'bigint'),
 ('id_vessel', 'bigint'),
 ('lat', 'float'),
 ('lon', 'float'),
 ('timestamp_pos', 'bigint')]

In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType, ArrayType
filtered4 = dataframe_temp.filter(calculateDistanceInKilometer(dataframe_temp.my_lat, dataframe_temp.my_lon, dataframe_temp.lat, dataframe_temp.lon) < 1.0)
filtered4.show()

+------------+---------+---------+----------------+---------+---------+---------+-------------+
my_id_vessel| my_lat| my_lon|my_timestamp_pos|id_vessel| lat| lon|timestamp_pos|
+------------+---------+---------+----------------+---------+---------+---------+-------------+
 28052|18.437849|-93.18948| 1416007969000| 6750|18.433628|-93.18375|1416007969000|
 28052|18.437849|-93.18948| 1416007969000| 85833| 18.432|-93.18712|1416007969000|
 28052|18.437849|-93.18948| 1416007969000| 151508|18.432825|-93.18878|1416007969000|
+------------+---------+---------+----------------+---------+---------+---------+-------------+